# `Nuqleon.IO.StreamSegment`

Provides a type providing a view into a `System.IO.Stream`.

## Reference the library

### Option 1 - Use a local build

If you have built the library locally, run the following cell to load the latest build.

In [1]:
#r "bin/Debug/net50/Nuqleon.IO.StreamSegment.dll"

### Option 2 - Use NuGet packages

If you want to use the latest published package from NuGet, run the following cell.

In [1]:
#r "nuget:Nuqleon.IO.StreamSegment,*-*"

## (Optional) Attach a debugger

If you'd like to step through the source code of the library while running samples, run the following cell, and follow instructions to start a debugger (e.g. Visual Studio). Navigate to the source code of the library to set breakpoints.

In [1]:
System.Diagnostics.Debugger.Launch();

## `StreamSegment`

A `StreamSegment` implements `Stream` by providing a view over an underlying `Stream`, starting at a specified offset, and with a specified length. An example is shown below, based on the idea of a length-prefixed encoding for sections of data, referred to as chunks.

First, let's create a function to create a stream that holds a number of chunks. Our goal is to use `StreamSegment` to reconstruct the chunks later.

In [1]:
using System.IO;

static Stream GetStream()
{
    var data = new MemoryStream();

    var chunks = new byte[][]
    {
        new byte[7] { 2, 3, 5, 11, 13, 17, 19 },
        new byte[3] { 23, 29, 31 },
    };

    using (var writer = new BinaryWriter(data, Encoding.UTF8, leaveOpen: true))
    {
        foreach (var chunk in chunks)
        {
            writer.Write(chunk.Length);
            writer.Write(chunk);
        }
    }

    data.Position = 0;

    return data;
}

The resulting stream contains data like this:

```
 07 00 00 00 02 03 05 0B 0D 11 13 03 00 00 00 17 1D 1F
 <---------> <------------------> <---------> <------>
```

where the two chunks are prefixed by the length. Let's print this to confirm the contents of the stream below.

In [1]:
var ms = GetStream();

var sb = new StringBuilder();

using (var reader = new BinaryReader(ms))
{
    int b;
    while ((b = reader.Read()) >= 0)
    {
        sb.Append($"{b:X2} ");
    }

    Console.WriteLine(sb.ToString());
}

07 00 00 00 02 03 05 0B 0D 11 13 03 00 00 00 17 1D 1F 


Now, let's implement the reader to reconstruct the chunks. In here, we'll read the length prefix from the original stream, followed by constructing a `StreamSegment` based on the retrieved length. We can then safely hand out this `Stream` instance to another utility (e.g. a deserializer) without running the risk of it reading out of bounds.

First, we'll create the utility that reads a chunk using a `Stream`-based input. In more realistic scenarios, this may invoke a deserializer.

In [1]:
static byte[] GetChunk(Stream stream)
{
    var res = new byte[stream.Length];
    stream.Read(res, 0, res.Length);
    return res;
}

We're now ready to read the chunks from the original stream using `StreamSegment`s over the chunks.

In [1]:
var ms = GetStream();

var chunks = new List<byte[]>();

using (var reader = new BinaryReader(ms))
{
    while (ms.Position < ms.Length)
    {
        // Read the length prefix.
        var length = reader.ReadInt32();

        // Construct a view into the underlying stream.
        using (var segment = new StreamSegment(ms, ms.Position, length))
        {
            chunks.Add(GetChunk(segment));
        }
    }
}

foreach (var chunk in chunks)
{
    Console.WriteLine(string.Join(", ", chunk));
}

2, 3, 5, 11, 13, 17, 19


23, 29, 31
